In [1]:
pip install pyet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from pyet.radiation import hargreaves,fao_24
from pyet.combination import pm_fao56
from pyet.temperature import hamon
from pyet.meteo_utils import extraterrestrial_r
import pandas as pd
import glob

In [10]:
DEG_TO_RAD_CONSTANT = 0.01745329251
KMH_TO_MS_CONSTANT = 1/3.6
MBAR_TO_KPA_CONSTANT = 1/10
lat = -33.3971255
elevation = 482.0
lat_rad = float(lat) *  DEG_TO_RAD_CONSTANT
RUTA = '/content/drive/MyDrive/Universidad/Evapotranspiration-Prediction-2022/data_processing/'

In [11]:
datasets = ['rh_minmax','t_minmax','temp-wind-rs-rh-pressure','et0_ref','rh_minmax_05-25','t_minmax_05-25','temp-wind-rs-rh-pressure_05-25','et0-real-05062022--25062022']
df_new = pd.DataFrame()
df_new_2 = pd.DataFrame()
for i in datasets:
  df = pd.read_csv(RUTA+"data/"+i+".csv",quotechar="'",keep_default_na=True)
  df.date = pd.to_datetime(df.date,format="%d-%m-%Y")
  df = df.set_index("date")
  df = df.interpolate()
  df = df.resample(rule='D').mean().copy()
  if i == 'rh_minmax':
    df_new = pd.concat([df_new,df[["rhmin","rhmax"]]],axis=1)
  elif i == 't_minmax':
    df_new = pd.concat([df_new,df[["tmin","tmax"]]],axis=1)
  elif i == 'temp-wind-rs-rh-pressure':
    df_new = pd.concat([df_new,df[["temp","wind","rs","rh","pressure"]]],axis=1)
  elif i == 'et0_ref':
    df_new = pd.concat([df_new,df[["et0"]]],axis=1)

for i in datasets:
  df = pd.read_csv(RUTA+"data/"+i+".csv",quotechar="'",keep_default_na=True)
  df.date = pd.to_datetime(df.date,format="%d-%m-%Y")
  df = df.set_index("date")
  df = df.interpolate()
  df = df.resample(rule='D').mean().copy()
  if i == 'rh_minmax_05-25':
    df_new_2 = pd.concat([df_new_2,df[["rhmin","rhmax"]]],axis=1)
  elif i == 't_minmax_05-25':
    df_new_2 = pd.concat([df_new_2,df[["tmin","tmax"]]],axis=1)
  elif i == 'temp-wind-rs-rh-pressure_05-25':
    df_new_2 = pd.concat([df_new_2,df[["temp","wind","rs","rh","pressure"]]],axis=1)
  elif i == 'et0-real-05062022--25062022':
    df_new_2 = pd.concat([df_new_2,df[["et0"]]],axis=1)
df_new = pd.concat([df_new,df_new_2],axis=0)
df_new
df_2 = df_new.copy()
df_2["ra"] = extraterrestrial_r(df_2.index,lat_rad)
df_2["et0_pmfao56"] = pm_fao56(df_2.temp , df_2.wind * KMH_TO_MS_CONSTANT , rs = df_2.rs , rh = df_2.rh , tmax = df_2.tmax , tmin = df_2.tmin , pressure = df_2.pressure * MBAR_TO_KPA_CONSTANT , lat = lat_rad , elevation = elevation , rhmin = df_2.rhmin , rhmax = df_2.rhmax)
df_2["et0_hs"] = hargreaves(tmean = df_2.temp , tmin = df_2.tmin , tmax = df_2.tmax , lat = lat_rad)
df_2["et0_fao24"] = fao_24(tmean = df_2.temp , wind = df_2.wind * KMH_TO_MS_CONSTANT , rs = df_2.rs , rh = df_2.rh , pressure = df_2.pressure * MBAR_TO_KPA_CONSTANT , elevation = elevation)
df_2["et0_hamon"] = hamon(tmean = df_2.temp , lat = lat_rad)

In [12]:
# Quitando la hora a las fechas para cada dataset
csv_files = glob.glob(RUTA+'data/pudahuel_weather_conditions-*')
csv_files.sort()
csv_files
dfs = []
for i in csv_files:
  wc = pd.read_csv(i)
  wc.date = pd.to_datetime(wc.date,format="%Y-%m-%d")
  for j in range (len(wc.date)):
    wc.loc[j, ('date')] = wc.date[j].strftime('%Y-%m-%d')
  wc = wc.set_index('date')
  dfs.append(wc)

In [13]:
# Asignación de valores numéricos a las condiciones climáticas, según el % de nubes en el aire
dfs[0].rename(columns={'clouds':'clouds9',
                        'weather_condition':'weather_condition9'},
               inplace=True)
dfs[1].rename(columns={'clouds':'clouds12',
                        'weather_condition':'weather_condition12'},
               inplace=True)
dfs[2].rename(columns={'clouds':'clouds16',
                        'weather_condition':'weather_condition16'},
               inplace=True)
dfs[3].rename(columns={'clouds':'clouds19',
                        'weather_condition':'weather_condition19'},
               inplace=True)
data = pd.concat([dfs[0],dfs[1],dfs[2],dfs[3]],axis=1)
data_new = pd.DataFrame()
data_new['clouds'] = (data['clouds9']+data['clouds12']+data['clouds16']+data['clouds19'])/4
data_new['weather_condition'] = data['weather_condition9']+data['weather_condition12']+data['weather_condition16']+data['weather_condition19']
data_new['cod_wc'] = 1-(data_new['clouds']/100)
data_new = data_new[['cod_wc']]

In [14]:
global_data = pd.concat([df_2,data_new],axis=1)
global_data.to_csv(RUTA+'data/PudahuelData-01012021-25062022.csv')